In [1]:
import json
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import CIFAR10
from PIL import Image, ImageDraw, ImageOps
import random

from transformers import CLIPProcessor, CLIPModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/openai/clip-vit-base-patch32/resolve/main/merges.txt: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out.
Trying to resume download...


merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [3]:
import pandas as pd

categories_dict = {"сумки": "bags", "столы": "tables", "одежда для девочек": "clothes for girls", "стулья": "chairs"}
colors_dict = {"zelenyi": "green", "chernyi": "black", "belyi": "white", "bordovyi": "burgundy", "krasnyi": "red", "bezhevyi": "beige", "raznocvetnyi": "multicolored", "rozovyi": "pink", "serebristyi": "silver", "korichnevyi": "brown", "fioletovyi": "purple", "seryi": "gray", "goluboi": "light blue", "oranzhevyi": "orange", "sinii": "blue", "biryuzovyi": "turquoise", "zolotoi": "gold", "zheltyi": "yellow"}

df = pd.read_csv('/kaggle/input/avitohack/dataset_colors/train_data.csv')
df['text'] = [f'{colors_dict[i[2]]} {categories_dict[i[1]]}' for i in df.values]
df['path'] = [f'/kaggle/input/avitohack/dataset_colors/train_data/{i[0]}.jpg' for i in df.values]
df

,id,category,target,text,path
0,11624739720,сумки,zelenyi,green bags,/kaggle/input/avitohack/dataset_colors/train_d...
1,11967354953,сумки,chernyi,black bags,/kaggle/input/avitohack/dataset_colors/train_d...
2,19026830493,сумки,belyi,white bags,/kaggle/input/avitohack/dataset_colors/train_d...
3,23088722867,сумки,chernyi,black bags,/kaggle/input/avitohack/dataset_colors/train_d...
4,23546690270,столы,belyi,white tables,/kaggle/input/avitohack/dataset_colors/train_d...
...,...,...,...,...,...
33298,37599243178,одежда для девочек,raznocvetnyi,multicolored clothes for girls,/kaggle/input/avitohack/dataset_colors/train_d...
33299,37599258032,сумки,bezhevyi,beige bags,/kaggle/input/avitohack/dataset_colors/train_d...
33300,37599262080,одежда для девочек,bezhevyi,beige clothes for girls,/kaggle/input/avitohack/dataset_colors/train_d...
33301,37599266008,одежда для девочек,bezhevyi,beige clothes for girls,/kaggle/input/avitohack/dataset_colors/train_d...


In [4]:
# Function to resize image to the required size
def resize_image(image, size):
    return image.resize(size, Image.LANCZOS)

class CIFAR10ClipDataset(Dataset):
    def __init__(self, cifar_dataset, num_samples=None, transform=None):
        self.cifar_dataset = cifar_dataset
        self.transform = transform
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples if self.num_samples else len(self.cifar_dataset)

    def __getitem__(self, idx):
        if self.num_samples:
            idx1 = random.randint(0, len(self.cifar_dataset) - 1)
            idx2 = random.randint(0, len(self.cifar_dataset) - 1)
        else:
            idx1 = idx
            idx2 = random.randint(0, len(self.cifar_dataset) - 1)

        
        img1, label1 = self.cifar_dataset.iloc[idx1][['path', 'text']]
        img2, label2 = self.cifar_dataset.iloc[idx2][['path', 'text']]

        img1, img2 = Image.open(img1).convert('RGB'), Image.open(img2).convert('RGB')

        img1, img2 = self.resize_and_pad(img1), self.resize_and_pad(img2)

        img1, img2 = self.transform(img1), self.transform(img2)

        # Determine if the images are from the same category
        if label1 == label2:
            return img1, img2, torch.tensor(1)  # Pair from the same category
        else:
            return img1, img2, torch.tensor(0)  # Pair from different categories

    def resize_and_pad(self, image, target_size=224):
        original_width, original_height = image.size
        aspect_ratio = original_width / original_height

        if aspect_ratio > 1:
            new_width = target_size
            new_height = int(target_size / aspect_ratio)
        else:
            new_height = target_size
            new_width = int(target_size * aspect_ratio)

        image = image.resize((new_width, new_height))

        pad_width = (target_size - new_width) // 2
        pad_height = (target_size - new_height) // 2

        padding = (pad_width, pad_height, target_size - new_width - pad_width, target_size - new_height - pad_height)
        image = ImageOps.expand(image, padding, fill=(0, 0, 0))

        return image

    

transform = transforms.Compose([
    transforms.ToTensor(),           # Convert images to PyTorch tensors
])





In [ ]:
import torch.optim as optim
from tqdm import tqdm

cifar_clip_dataset = CIFAR10ClipDataset(df, num_samples=128*50, transform=transform)
batch_size = 128
dataloader = DataLoader(cifar_clip_dataset, batch_size=batch_size, shuffle=True)

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, emb1, emb2, labels):
        similarities = torch.nn.functional.cosine_similarity(emb1, emb2)
        
        loss = torch.mean((1 - labels) * similarities + labels * (1 - similarities))
        return loss

criterion = CustomLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-6)

model.to(device)

epochs = 1

for epoch in range(epochs):
    running_loss = 0.0
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f"Epoch [{epoch+1}/{epochs}]") as tepoch:
        for i, data in tepoch:
            img1, img2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
            
            emb1 = model.get_image_features(img1)
            emb2 = model.get_image_features(img2)
            
            loss = criterion(emb1, emb2, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item())
            running_loss += loss.item()
            
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.3f}")

In [19]:
torch.save(model.state_dict(), "clip_tuned_by_similarity.pt")

In [27]:
class TestDataaste(Dataset):
    def __init__(self, cifar_dataset, num_samples=None, transform=None):
        self.cifar_dataset = cifar_dataset
        self.transform = transform
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples if self.num_samples else len(self.cifar_dataset)

    def __getitem__(self, idx1):

        
        img1, label1 = self.cifar_dataset.iloc[idx1][['path', 'text']]

        img1 = Image.open(img1).convert('RGB')

        img1= self.resize_and_pad(img1)

        img1 = self.transform(img1)
   
        return img1, label1

    def resize_and_pad(self, image, target_size=224):
        original_width, original_height = image.size
        aspect_ratio = original_width / original_height

        if aspect_ratio > 1:
            new_width = target_size
            new_height = int(target_size / aspect_ratio)
        else:
            new_height = target_size
            new_width = int(target_size * aspect_ratio)

        image = image.resize((new_width, new_height))

        pad_width = (target_size - new_width) // 2
        pad_height = (target_size - new_height) // 2

        padding = (pad_width, pad_height, target_size - new_width - pad_width, target_size - new_height - pad_height)
        image = ImageOps.expand(image, padding, fill=(0, 0, 0))

        return image

In [31]:
import torch
from sklearn.metrics import accuracy_score

def evaluate_accuracy(model, test_loader, device="cuda"):
    model.to(device)
    model.eval()  # Переводим модель в режим оценки
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images) 
            _, preds = torch.max(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

In [ ]:
cifar10_classes = df.text.unique()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


cifar_clip_dataset = TestDataaste(df.sample(1000),transform=transform)
test_loader = DataLoader(cifar_clip_dataset, batch_size=64, shuffle=False)


accuracy = evaluate_accuracy(model, test_loader, device="cuda")
print(f"Accuracy: {accuracy * 100:.2f}%")